In [11]:
import pandas
import re
import json
import pathlib
from google_comments import MEDIA_PATH

In [12]:
files = MEDIA_PATH.joinpath('concurrence_aprium').glob('**/*.json')

In [13]:
files_data = []

for file in files:
    with open(file, mode='r', encoding='utf-8') as f:
        data = json.load(f)
        data[0]['file_name'] = pathlib.Path(f.name).name
        files_data.extend(data)

In [14]:
places = []
for item in files_data:
    file_places = item['places']
    for place in file_places:
        place['file_name'] = item['file_name']
    places.extend(file_places)

In [15]:
df = pandas.DataFrame(data=places)

In [21]:
df = df[['name', 'latitude', 'longitude', 'address', 'rating', 'number_of_reviews', 'file_name']]
final_df = df[['name', 'latitude', 'longitude',
               'address', 'rating', 'number_of_reviews']]

In [17]:
no_address = df[df['address'].isna()]
no_address.count()

name                 71
latitude             71
longitude            71
address               0
rating               71
number_of_reviews    71
file_name            71
dtype: int64

In [18]:
def zip_code(value):
    if value is None:
        return None
    result = re.search(r'(\d{5})', value)
    if result:
        return result.group(1)
    return None


def city(value):
    if value is None:
        return None
    result = re.search(r'\d{5}\s?(.*)', value)
    if result:
        return result.group(1)
    return None

df['zip_code'] = df['address'].apply(zip_code)        
df['city'] = df['address'].apply(city)        

In [19]:
df['is_duplicated'] = df.duplicated(subset=['address'])
duplicates = df[df['is_duplicated'] == True & ~df['address'].isna()]
duplicates = duplicates.sort_values('address')
# duplicates.to_csv('duplicated.csv', index=False)

In [ ]:
final_df.to_csv(MEDIA_PATH / 'concurrence.csv', encoding='utf-8', index=False)